# Panle Regression - Firm Characteristics

### Random Effects Panel Regression

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import sklearn
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects


Data

In [2]:
df = pd.read_csv("Dataframes/macro_2.csv")
df["Earnings Per Share - Actual Surprise AbsVals"] = df["Earnings Per Share - Actual Surprise"].abs()
df = df.loc[:, ["Instrument", "Date", "GICS Industry Group Name", "Earnings Per Share - Actual Surprise", "Earnings Per Share - Actual Surprise AbsVals", "WACC Inflation Adjusted Risk Free Rate, (%)", "Inflation Premium(%)", "90-Day AA Financial Commercial Paper Interest Rate", "CPFFM", "CBOE Crude Oil ETF Volatility Index", "Consumer Sentiment", "Unemployment rate"]]
df

FileNotFoundError: [Errno 2] No such file or directory: 'Dataframes/macro_2.csv'

In [3]:
df["Date"] = pd.to_datetime(df["Date"])

Remove outliers

In [4]:
# Calculate the 0.15th and 99.85th percentile
pct_0_15 = np.percentile(df['Earnings Per Share - Actual Surprise'], 0.5)
pct_99_85 = np.percentile(df['Earnings Per Share - Actual Surprise'], 99.5)

# Remove the data points outside the 0.15th and 99.85th percentile
df_clean = df[(df['Earnings Per Share - Actual Surprise'] >= pct_0_15) & (df['Earnings Per Share - Actual Surprise'] <= pct_99_85)].copy()

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,"WACC Inflation Adjusted Risk Free Rate, (%)",Inflation Premium(%),90-Day AA Financial Commercial Paper Interest Rate,CPFFM,CBOE Crude Oil ETF Volatility Index,Consumer Sentiment,Unemployment rate
0,AVY.N,2013-01-01,Materials,11.178,11.178,NaN,0.359537,0.17,0.02,22.54,76.67,7.733333
1,AVY.N,2013-04-01,Materials,2.482,2.482,NaN,0.360572,0.14,0.03,23.47,81.67,7.533333
2,AVY.N,2013-07-01,Materials,1.068,1.068,NaN,0.463789,0.13,0.04,24.10,81.57,7.233333
3,AVY.N,2013-10-01,Materials,8.095,8.095,NaN,0.441854,0.13,0.04,19.74,76.93,6.933333
4,AVY.N,2014-01-01,Materials,1.471,1.471,NaN,0.465460,0.13,0.05,19.56,80.93,6.666667
...,...,...,...,...,...,...,...,...,...,...,...,...
20115,POOL.OQ,2021-10-01,Retailing,17.194,17.194,1.527139,0.382645,0.14,0.06,43.04,69.90,4.200000
20116,POOL.OQ,2022-01-01,Retailing,40.267,40.267,1.515266,0.380946,0.47,0.32,52.70,63.13,3.800000
20117,POOL.OQ,2022-04-01,Retailing,34.342,34.342,2.325202,0.495588,1.39,0.61,50.46,57.87,3.600000
20118,POOL.OQ,2022-07-01,Retailing,1.503,1.503,3.092855,0.423045,2.89,0.64,49.86,56.10,3.566667


In [ ]:
#removing oly 99th percentile outliers
'''
# Calculate the 99th percentile
pct_99 = np.percentile(df['Earnings Per Share - Actual Surprise'], 99)

# Remove the data points above the 99th percentile
df_clean = df[df['Earnings Per Share - Actual Surprise'] <= pct_99].copy()

df_clean
'''

Standardising

get wierd results when standardising

In [5]:
rescale = df_clean
#rescale["Earnings Per Share - Actual Surprise"] = MinMaxScaler().fit_transform(np.array(rescale["Earnings Per Share - Actual Surprise"]).reshape(-1,1))
rescale["WACC Inflation Adjusted Risk Free Rate, (%)"] = MinMaxScaler().fit_transform(np.array(rescale["WACC Inflation Adjusted Risk Free Rate, (%)"]).reshape(-1,1))
rescale["Inflation Premium(%)"] = MinMaxScaler().fit_transform(np.array(rescale["Inflation Premium(%)"]).reshape(-1,1))
rescale["90-Day AA Financial Commercial Paper Interest Rate"] = MinMaxScaler().fit_transform(np.array(rescale["90-Day AA Financial Commercial Paper Interest Rate"]).reshape(-1,1))
rescale["CPFFM"] = MinMaxScaler().fit_transform(np.array(rescale["CPFFM"]).reshape(-1,1))
rescale["CBOE Crude Oil ETF Volatility Index"] = MinMaxScaler().fit_transform(np.array(rescale["CBOE Crude Oil ETF Volatility Index"]).reshape(-1,1))
rescale["Consumer Sentiment"] = MinMaxScaler().fit_transform(np.array(rescale["Consumer Sentiment"]).reshape(-1,1))
rescale["Unemployment rate"] = MinMaxScaler().fit_transform(np.array(rescale["Unemployment rate"]).reshape(-1,1))
df_clean = rescale
df_clean

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,"WACC Inflation Adjusted Risk Free Rate, (%)",Inflation Premium(%),90-Day AA Financial Commercial Paper Interest Rate,CPFFM,CBOE Crude Oil ETF Volatility Index,Consumer Sentiment,Unemployment rate
0,AVY.N,2013-01-01,Materials,11.178,11.178,NaN,0.552998,0.016949,0.150685,0.058466,0.480271,0.443262
1,AVY.N,2013-04-01,Materials,2.482,2.482,NaN,0.555632,0.009685,0.164384,0.069128,0.597011,0.421986
2,AVY.N,2013-07-01,Materials,1.068,1.068,NaN,0.818318,0.007264,0.178082,0.076350,0.594677,0.390071
3,AVY.N,2013-10-01,Materials,8.095,8.095,NaN,0.762495,0.007264,0.178082,0.026367,0.486341,0.358156
4,AVY.N,2014-01-01,Materials,1.471,1.471,NaN,0.822572,0.007264,0.191781,0.024304,0.579734,0.329787
...,...,...,...,...,...,...,...,...,...,...,...,...
20115,POOL.OQ,2021-10-01,Retailing,17.194,17.194,0.456602,0.611807,0.009685,0.205479,0.293477,0.322204,0.067376
20116,POOL.OQ,2022-01-01,Retailing,40.267,40.267,0.453705,0.607483,0.089588,0.561644,0.404219,0.164137,0.024823
20117,POOL.OQ,2022-04-01,Retailing,34.342,34.342,0.651306,0.899246,0.312349,0.958904,0.378539,0.041326,0.003546
20118,POOL.OQ,2022-07-01,Retailing,1.503,1.503,0.838591,0.714626,0.675545,1.000000,0.371661,0.000000,0.000000


In [6]:
df_panel = df_clean.copy()

#### Fixed Effects

In a fixed effects panel regression, the individual-specific effects are modeled as fixed variables that do not vary across time. This means that the coefficients of the independent variables are estimated based on the within-entity variation in the data, which eliminates the effect of time-invariant unobserved heterogeneity.

Fixed effects models are useful when there are time-invariant unobserved variables that may affect the dependent variable, but are not included in the model. By modeling the individual-specific effects as fixed variables, fixed effects models can control for this unobserved heterogeneity and estimate the coefficients of the independent variables based on the within-entity variation, which provides more efficient estimates of the coefficients.

One limitation of fixed effects models is that they do not allow for testing the effect of time-invariant variables on the dependent variable. In addition, fixed effects models may suffer from the incidental parameter problem, which may lead to biased estimates of the coefficients of the independent variables in the presence of a large number of fixed effects.

In [7]:
df_clean['Instrument'] = df_clean['Instrument'].astype('category')
df_clean['Date'] = pd.to_datetime(df_clean['Date'])

# set the index to be the time variable and the cross-sectional variable
df_clean.set_index(['Instrument', 'Date'], inplace=True)

Absolute Surprise Values

In [12]:
y = df_clean.loc[:, "Earnings Per Share - Actual Surprise AbsVals"]
X = df_clean.loc[:, "Inflation Premium(%)":]

# perform the fixed effects panel regression
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary

/Users/ginevrabramante/opt/anaconda3/envs/AnalystForecast/lib/python3.11/site-packages/linearmodels/panel/model.py:1183: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


Dep. Variable:,Earnings Per Share - Actual Surprise AbsVals,R-squared:,3.064e-07
Estimator:,PanelOLS,R-squared (Between):,-1.449e+22
No. Observations:,18608,R-squared (Within):,-3.538e+21
Date:,"Thu, Mar 02 2023",R-squared (Overall):,-1.062e+22
Time:,18:42:44,Log-likelihood,-7.294e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,0.0009
Entities:,502,P-value,1.0000
Avg Obs:,37.068,Distribution:,"F(6,18061)"
Min Obs:,1.0000,,
Max Obs:,40.000,F-statistic (robust):,0.0009


By Industry: example tech industry

Absolute Surprise Values

In [14]:
group = df_panel[df_panel['GICS Industry Group Name'] == 'Technology Hardware & Equipment']

group['Instrument'] = group['Instrument'].astype('category')
group['Date'] = pd.to_datetime(group['Date'])
group.set_index(['Instrument', 'Date'], inplace=True)

y = group.loc[:, "Earnings Per Share - Actual Surprise AbsVals"]
X = group.loc[:, "Inflation Premium(%)":]

model = PanelOLS(y, X, entity_effects=True, time_effects=True).fit()
model

/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_18906/1739877626.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['Instrument'] = group['Instrument'].astype('category')
/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_18906/1739877626.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['Date'] = pd.to_datetime(group['Date'])
/Users/ginevrabramante/opt/anaconda3/envs/AnalystForecast/lib/python3.11/site-packages/linearmodels/panel/model.py:1183: MissingValueWarning: 
Inputs

Dep. Variable:,Earnings Per Share - Actual Surprise AbsVals,R-squared:,0.0000
Estimator:,PanelOLS,R-squared (Between):,-0.6845
No. Observations:,729,R-squared (Within):,-2.1084
Date:,"Thu, Mar 02 2023",R-squared (Overall):,-1.3433
Time:,18:45:28,Log-likelihood,-2614.9
Cov. Estimator:,Unadjusted,,
,,F-statistic:,0.0000
Entities:,19,P-value,1.0000
Avg Obs:,38.368,Distribution:,"F(6,665)"
Min Obs:,28.000,,
Max Obs:,40.000,F-statistic (robust):,4.165e-31
